## Imports

In [2]:
import pandas as pd
import sqlalchemy as sa
import sqlite3
import openai
import pyodbc
import pypyodbc as odbc

## Database credentials

In [3]:
''''
server='like4.database.windows.net'
database='like'
connection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:like4.database.windows.net,1433;Database=like;Uid=CloudSA42bee827;Pwd={XP2U@X3R5EOQ23};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
conn=odbc.connect(connection_string)
'''

"'\nserver='like4.database.windows.net'\ndatabase='like'\nconnection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:like4.database.windows.net,1433;Database=like;Uid=CloudSA42bee827;Pwd={XP2U@X3R5EOQ23};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'\nconn=odbc.connect(connection_string)\n"

In [4]:
''''
sql=''' 
#SELECT count(user_id)
#from dbo.Users 
'''
cursor=conn.cursor()
cursor.execute(sql)
dataset=cursor.fetchall()
print(dataset)
'''

'\ncursor=conn.cursor()\ncursor.execute(sql)\ndataset=cursor.fetchall()\nprint(dataset)\n'

## Connecting to the data warehouse

In [5]:
f = open("Credintials.txt", "r")
email = None
password = None
for i, line in enumerate(f):
    if i == 0:
        email = line.strip()  # Assign the first line to email variable
    elif i == 1:
        password = line.strip()  # Assign the second line to password variable

f.close()

In [6]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= email # use %40 inplace of @
pass1=password # use %40 inplace of @   

# Create a connection string for SQL Server using Windows authentication
connection_string = f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'

In [7]:
engine = sa.create_engine(connection_string, echo=True, connect_args={'autocommit': True}, fast_executemany=True)


## Data Preprocessing

In [74]:
sql_query = '''
SELECT 
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_NAME IN (SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE')
    and TABLE_NAME like 'dim_products' 
    or TABLE_NAME like 'fact_trans%' 
    or TABLE_NAME like 'dim_allusers%'
    or TABLE_NAME like 'dim_stores%' 
    
'''
df = pd.read_sql(sql_query, engine)

2024-01-09 12:46:25,212 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-09 12:46:25,212 INFO sqlalchemy.engine.Engine [cached since 775.4s ago] ('BASE TABLE', "\nSELECT \n    TABLE_SCHEMA,\n    TABLE_NAME,\n    COLUMN_NAME,\n    DATA_TYPE\nFROM \n    INFORMATION_SCHEMA.COLUMNS\nWHERE \n    TABLE_NAME IN (SEL ... (96 characters truncated) ... ME like 'dim_products' \n    or TABLE_NAME like 'fact_trans%' \n    or TABLE_NAME like 'dim_allusers%'\n    or TABLE_NAME like 'dim_stores%' \n    \n", 'dbo')
2024-01-09 12:46:27,344 INFO sqlalchemy.engine.Engine 
SELECT 
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_NAME IN (SEL

In [75]:
table_schema=df.loc[0][0]

In [76]:
print(table_schema)

dbo


In [77]:
df=df.drop(columns=['TABLE_SCHEMA'])


In [78]:
df.columns

Index(['TABLE_NAME', 'COLUMN_NAME', 'DATA_TYPE'], dtype='object')

In [79]:
df_dict = df.to_dict(orient='records')
df_dict

[{'TABLE_NAME': 'dim_allusers', 'COLUMN_NAME': 'user_id', 'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'first_name',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'last_name',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'email',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'store_country_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'customer_group_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'phone',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'representative_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'createdDate',
  'DATA_TYPE': 'date'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'created_on',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'dim_allusers',
  'COLUMN_NAME': 'createdDateKey',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'dim_allus

In [80]:
unique_table_names = set(entry['TABLE_NAME'] for entry in df_dict)

# Display unique table names
for table_name in unique_table_names:
    print(table_name)


dim_products
dim_allusers
fact_transactions
dim_stores


In [173]:
import os
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"
                       

In [202]:
query = None
def generate_sql_query(dictionary, text, table_schema):
    prompt = """**Task:** Generate a concise SQL Server query using only function calls and parentheses.

**Context:**
- Available tables, columns, and data types: {}
- Table schemas: {}

**Requirements:**
- **Translate natural language input:** Accurately convert the given text into a corresponding SQL query.
- **Concise language:** Use only function calls and parentheses, omitting descriptive words.
- **Example:** Replace "DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)" with "DATE_SUB(CURRENT_DATE(), 1)".
- **Handle joins:** Identify and join necessary tables as required by the input text.
    Input: {} SQL Query (without descriptive words).
    """.format(dictionary, table_schema, text)

    request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        stop=None,

    )
    global query
    query = request.choices[0].message.content
    return query


In [203]:
def executeSQLquery(df_dict,text,table_schema ,max_retries):
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            query = generate_sql_query(df_dict,text,table_schema)
            result = pd.read_sql(query,engine)
            return result
        except Exception as e:
            print(f"Error executing query: {e}")
            retry_count += 1
            if retry_count < max_retries:
                print(f"Retrying... (Attempt {retry_count}/{max_retries})")
            else:
                print("Max retry attempts reached. Exiting.")
                raise

In [256]:
def get_result_prompt(question, query, df_dict, table_schema, queryResult):

    prompt = """**Task:** Answer the input question in natural language based on the provided data.

**Context:**
- Table schemas: {}
- Input SQL query (with potential joins): {}
- Query result: {}
- Input question: {}

**Requirements:**
- Generate a human-readable response that directly answers the question.
- Avoid mentioning any SQL queries or table names in the response.
- Handle empty results with "No data found."
- Gracefully handle errors with "Try it in another way."

**Example:**
 What were the total sales in January 2023?
**Response:** The total sales in January 2023 were 123 million dollars.
""".format(table_schema, query, queryResult, question)
    return prompt
text = 'top 5 products in terms of amount'
queryResult = executeSQLquery(df_dict,text,table_schema ,6)
result_prompt = get_result_prompt(text,query,df_dict,table_schema,queryResult)
print(result_prompt)

2024-01-09 16:11:52,974 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-09 16:11:52,974 INFO sqlalchemy.engine.Engine [cached since 1.311e+04s ago] ('BASE TABLE', 'SELECT TOP 5 product_id, SUM(amount) as total_amount\nFROM fact_transactions\nGROUP BY product_id\nORDER BY total_amount DESC;', 'dbo')
2024-01-09 16:11:53,889 INFO sqlalchemy.engine.Engine SELECT TOP 5 product_id, SUM(amount) as total_amount
FROM fact_transactions
GROUP BY product_id
ORDER BY total_amount DESC;
2024-01-09 16:11:53,889 INFO sqlalchemy.engine.Engine [raw sql] ()
**Task:** Answer the input question in natural language based on the provided data.

**Context:**
- Table schemas: dbo
- Input SQL query (with potential

In [257]:

request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": result_prompt},
        ],
        stop=None,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
    )
answer = request.choices[0].message.content
answer


'The top 5 products in terms of amount are product 54 with a total amount of 291.7 million dollars, product 1628 with a total amount of 230.9 million dollars, product 975 with a total amount of 130.5 million dollars, product 1627 with a total amount of 124 million dollars, and product 53 with a total amount of 107.3 million dollars.'

In [261]:
def graph(question,query,df_dict,table_schema,queryResult):

    prompt = """**Task:** Write Python code using pandas and plotly **without any comments** to generate the graph based on the provided data.

**Context:**
- Table schema: {}
- Input SQL query (with potential joins): {}
- Query result: {}
- Input question: {}

**Requirements:**
- Handle empty results with "No data found."
- Gracefully handle errors with "Try it in another way."

**Example:**
> import pandas as pd
> import plotly
> 
> ...
> 
""".format(table_schema, query, queryResult, question)


    request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
                stop=None,
       
    )
    answer = request.choices[0].message.content
    return answer

graph_code = graph(text,query,df_dict,table_schema,queryResult)
graph_code
exec(graph_code)

In [262]:
graph_code

"import pandas as pd\nimport plotly.express as px\n\n# input query result\ndata = {'product_id': [54, 1628, 975, 1627, 53],\n        'total_amount': [2.916845e+08, 2.308801e+08, 1.305104e+08, 1.240185e+08, 1.072587e+08]}\ndf = pd.DataFrame(data)\n\n# create bar plot\nfig = px.bar(df, x='product_id', y='total_amount', title='Top 5 Products by Amount')\nfig.update_layout(xaxis_title='Product ID', yaxis_title='Total Amount')\nfig.show()"

In [221]:

request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": result_prompt},
        ],
        stop=None,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
    )
answer = request.choices[0].message.content
answer


'**Response:** The top 5 products sales in each country are as follows:\n\nIn Emirates:\n1. LikeCard Gift Card 20 SAR (KSA Account)\n2. LikeCard Gift Card 20 AED (UAE Account)\n3. Binance 1000 Tether ( USDT ) - Global\n\nIn Saudi Arabia:\n1. noon 1000 AED - (Shipping only in UAE)\n\nPlease note that there is no data available for other countries.'

In [214]:
text = 'top 5 prodycts sales in each country'
queryResult = executeSQLquery(df_dict,text,table_schema ,6)
result_prompt = get_result_prompt(text,query,df_dict,table_schema,queryResult)
request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": result_prompt},
        ],
        stop=None,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
    )
answer = request.choices[0].message.content
print(answer)
#exec(graph(text,query,df_dict,table_schema,queryResult))

2024-01-09 15:54:07,891 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-09 15:54:07,898 INFO sqlalchemy.engine.Engine [cached since 1.204e+04s ago] ('BASE TABLE', '(SELECT product_name, country, SUM(qty) as total_sales\nFROM fact_transactions\nJOIN dim_products ON fact_transactions.product_id = dim_products.product_id\nJOIN dim_stores ON fact_transactions.store_id = dim_stores.country_id\nGROUP BY product_name, country\nORDER BY total_sales DESC\nLIMIT 5)', 'dbo')
2024-01-09 15:54:08,621 INFO sqlalchemy.engine.Engine (SELECT product_name, country, SUM(qty) as total_sales
FROM fact_transactions
JOIN dim_products ON fact_transactions.product_id = dim_products.product_id
JOIN dim_stores ON f

ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near the keyword 'ORDER'. (156) (SQLExecDirectW)")
[SQL: (SELECT country, product_name, SUM(qty) AS sales
FROM fact_transactions
JOIN dim_products ON fact_transactions.product_id = dim_products.product_id
JOIN dim_stores ON fact_transactions.store_id = dim_stores.store_id
GROUP BY country, product_name
ORDER BY country, sales DESC)
]
(Background on this error at: https://sqlalche.me/e/14/f405)